In [ ]:
import numpy as np
from tvm import te
import tvm
from tvm import relay

In [ ]:
def example():
    shape = (1, 64, 54, 54)
    c_data = np.empty(shape).astype("float32")
    c = relay.const(c_data)
    weight = relay.var("weight", shape=(64, 64, 3, 3))
    x = relay.var("x", relay.TensorType((1, 64, 56, 56), "float32"))
    conv = relay.nn.conv2d(x, weight)
    y = relay.add(c, c)
    y = relay.multiply(y, relay.const(2, "float32"))
    y = relay.add(conv, y)
    z = relay.add(y, c)
    z1 = relay.add(y, c)
    z2 = relay.add(z, z1)
    return relay.Function([x, weight], z2)

In [ ]:
f = example()

f

In [ ]:
@relay.op.register_alter_op_layout('nn.conv2d',level=100)
def alter_conv2d(attrs, inputs, tinfos, ouput_type):
    data,weight = inputs
    new_attrs = dict(attrs)
    new_attrs["data_layout"] = "NCHW16c"

    return relay.nn.conv2d(data, weight, **new_attrs)

In [ ]:
f = example()
mod = tvm.IRModule.from_expr(f)
fold_const = relay.transform.FoldConstant()

mod = fold_const(mod)
mod

In [ ]:
f = example()
mod = tvm.IRModule.from_expr(f)
mod = relay.transform.EliminateCommonSubexpr()(mod)
mod.functions

In [ ]:
f = example()
mod = tvm.IRModule.from_expr(f)
mod = relay.transform.FuseOps(fuse_opt_level=0)(mod)
mod.functions

In [ ]:
f = example()

mod = tvm.IRModule.from_expr(f)
seq = tvm.transform.Sequential(
    [
        relay.transform.FoldConstant(),
        relay.transform.EliminateCommonSubexpr(),
        relay.transform.FuseOps(fuse_opt_level=2)
    ]
)

seq(mod)

mod.functions

In [ ]:
f = example()

mod = tvm.IRModule.from_expr(f)

# with tvm.transform.PassContext(opt_level=2):
#     seq(mod)
seq(mod)

mod.functions

In [ ]:
f = example()

mod = tvm.IRModule.from_expr(f)

with tvm.transform.PassContext(opt_level=3):
    seq(mod)

mod.functions

In [ ]:
@relay.transform.function_pass(opt_level=1)
class CustomPipeline:
    def __init__(self,mutier) -> None:
        self.mutier = mutier

    def transform_function(self,func,mod,ctx):
        obj = self

        class ReplaceConst(tvm.relay.ExprMutator):
            def visit_constant(self,c):
                return relay.multiply(c,obj.mutier)

        return ReplaceConst().visit(func)

In [ ]:
f = example()
mod = tvm.IRModule.from_expr(f)

cus = CustomPipeline(relay.const(4,'float32'))
mod = cus(mod)

mod

In [ ]:
f = example()

mod = tvm.IRModule.from_expr(f)
seq = tvm.transform.Sequential(
    [
        relay.transform.FoldConstant(),
        tvm.transform.PrintIR(),
        relay.transform.EliminateCommonSubexpr(),
        relay.transform.FuseOps(fuse_opt_level=2)
    ]
)

seq(mod)

mod.functions

In [ ]:
seq = tvm.transform.Sequential(
    [
        relay.transform.FoldConstant(),
        relay.transform.EliminateCommonSubexpr(),
        relay.transform.FuseOps(fuse_opt_level=2)
    ]
)

@tvm.instrument.pass_instrument
class PrintIR:
    """Print the name of the pass, the IR, only before passes execute."""

    def run_before_pass(self, mod, info):
        print("Running pass: {}", info)
        print(mod)


with tvm.transform.PassContext(opt_level=3, instruments=[PrintIR()]):
    with tvm.target.Target("llvm"):
        # Perform the optimizations.
        mod = seq(mod)
print(mod)

print("done")